# Model Tutorial

In [2]:
import qnn_gen as qg
import numpy as np

"Model" is one of the most overloaded words in mathematical fields. One could consider a quantum machine learning "model" to mean the end-to-end processes of encoding, choice of gate architecture and parameters, measurement, post-processing, and optimization method. 

In QNN-Gen, the `Model` class is more restricted. The `Model` class corresponds to various achitectures of gates and ansätze with learnable parameters. Typically, these layers act on a state that has already been encoded. 

## The Model base class

In `model.py` you can find the `Model` base class and serveral implemenatations which derive from the base class. You can see that the derived classes ought to have attributes for `n_qubits` and `parameters`, and must override the `circuit` function. 

Additionally, the derived classes can override the `default_measurement` function. It is often the case that the choice of model implies which measurements are sensible. Implementing this function allows you to combine and run circuits without explictly passing a `Measurement` object.

In [42]:
from abc import ABC, abstractmethod

class Model(ABC):
    """
    Class to define a QML model, whether it be a particular ansatz or a perceptron model.

    Abstract class. Derived classes overwrite circuit() and may overwrite default_measuremnt().
    Derived classes should ensure that the self.n_qubits and self.parameters attributes are updated.
    """

    def __init__(self):
        self.n_qubits = None
        self.parameters = None

    @abstractmethod
    def circuit():
        """
        Abstract method. Overwrite to return the circuit.

        Returns:
            - (qiskit.QuantumCircuit): The circuit defined by the model
        """
        pass
    
    def default_measurement():
        """
        Often, the model implies what measurements are sensible.

        Returns:
            - (Measurement): The default measurement for the model
        """

        raise NotImplementedError

Now we'll go over some of the derived classes:
- Tree Tensor Network
- Binary Perceptron
- Entangled Qubit

### Tree Tensor Network

A simple way to instantiate a Tree Tensor network:

In [43]:
ttn = qg.TreeTensorNetwork(n_qubits=8)
ttn_circuit = ttn.circuit()

print(ttn_circuit)
print(ttn.parameters)

     ┌─────────────┐                                             
q_0: ┤ RY(0.28935) ├──■──────────────────────────────────────────
     └┬────────────┤┌─┴─┐ ┌────────────┐                         
q_1: ─┤ RY(2.5025) ├┤ X ├─┤ RY(2.3083) ├──■──────────────────────
      ├────────────┤└───┘ └────────────┘  │                      
q_2: ─┤ RY(1.7661) ├──■───────────────────┼──────────────────────
      ├────────────┤┌─┴─┐ ┌────────────┐┌─┴─┐┌─────────────┐     
q_3: ─┤ RY(1.6991) ├┤ X ├─┤ RY(1.6802) ├┤ X ├┤ RY(0.99314) ├──■──
      ├────────────┤├───┤ ├────────────┤├───┤└┬────────────┤┌─┴─┐
q_4: ─┤ RY(1.0028) ├┤ X ├─┤ RY(2.7517) ├┤ X ├─┤ RY(1.2519) ├┤ X ├
      ├────────────┤└─┬─┘ └────────────┘└─┬─┘ └────────────┘└───┘
q_5: ─┤ RY(2.1347) ├──■───────────────────┼──────────────────────
      ├────────────┤┌───┐┌─────────────┐  │                      
q_6: ─┤ RY(1.8147) ├┤ X ├┤ RY(0.66316) ├──■──────────────────────
      ├────────────┤└─┬─┘└─────────────┘                         
q_7: ─┤ RY

If you don't pass arugments for the parameters of the rotation gates, they are randomly initialized. You may specificy them explictly using the `angles` argument:

In [44]:
angles = np.arange(1, 15)

ttn = qg.TreeTensorNetwork(n_qubits=8, angles=angles)
ttn_circuit = ttn.circuit()

print(ttn_circuit)
print(ttn.parameters)

     ┌───────┐                                   
q_0: ┤ RY(1) ├──■────────────────────────────────
     ├───────┤┌─┴─┐┌───────┐                     
q_1: ┤ RY(2) ├┤ X ├┤ RY(9) ├───■─────────────────
     ├───────┤└───┘└───────┘   │                 
q_2: ┤ RY(3) ├──■──────────────┼─────────────────
     ├───────┤┌─┴─┐┌────────┐┌─┴─┐┌────────┐     
q_3: ┤ RY(4) ├┤ X ├┤ RY(10) ├┤ X ├┤ RY(13) ├──■──
     ├───────┤├───┤├────────┤├───┤├────────┤┌─┴─┐
q_4: ┤ RY(5) ├┤ X ├┤ RY(11) ├┤ X ├┤ RY(14) ├┤ X ├
     ├───────┤└─┬─┘└────────┘└─┬─┘└────────┘└───┘
q_5: ┤ RY(6) ├──■──────────────┼─────────────────
     ├───────┤┌───┐┌────────┐  │                 
q_6: ┤ RY(7) ├┤ X ├┤ RY(12) ├──■─────────────────
     ├───────┤└─┬─┘└────────┘                    
q_7: ┤ RY(8) ├──■────────────────────────────────
     └───────┘                                   
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14]


User also may change the rotation gate and the entangling gate from their default values of `RY` and `CX`. For example, with the help of the `Gate` class in `utility.py`, a Tree Tensor Network with `RX` as the roation gate and a `CZ` as the two qubit gate:

In [46]:
ttn = qg.TreeTensorNetwork(n_qubits=8,
                           angles=angles, 
                           rotation_gate=qg.Gate.RX, 
                           entangling_gate=qg.Gate.CZ)
ttn_circuit = ttn.circuit()

print(ttn_circuit)

     ┌───────┐                             
q_0: ┤ RX(1) ├─■───────────────────────────
     ├───────┤ │ ┌───────┐                 
q_1: ┤ RX(2) ├─■─┤ RX(9) ├──■──────────────
     ├───────┤   └───────┘  │              
q_2: ┤ RX(3) ├─■────────────┼──────────────
     ├───────┤ │ ┌────────┐ │ ┌────────┐   
q_3: ┤ RX(4) ├─■─┤ RX(10) ├─■─┤ RX(13) ├─■─
     ├───────┤   ├────────┤   ├────────┤ │ 
q_4: ┤ RX(5) ├─■─┤ RX(11) ├─■─┤ RX(14) ├─■─
     ├───────┤ │ └────────┘ │ └────────┘   
q_5: ┤ RX(6) ├─■────────────┼──────────────
     ├───────┤   ┌────────┐ │              
q_6: ┤ RX(7) ├─■─┤ RX(12) ├─■──────────────
     ├───────┤ │ └────────┘                
q_7: ┤ RX(8) ├─■───────────────────────────
     └───────┘                             


### Binary Perceptron

### Entangled Qubit